In [1]:
from squad_solver import *
from utils_squads import *
from experiments_runner import run_experiment, repeat_run_experiment

params = {
    "budget": 100000,
    "number_of_defenders": 4,
    "number_of_midfielders": 3,
    "number_of_forwards": 3,
    "min_chemistry": 15,
    "M":11
}

general_components = "Avg_Global, PositionGK, PositionDEF, PositionMID, PositionFOR, Budget"
general_components += ", y, natCount, chemistryNat1, chemistryNat2, chemistryNat3"
formulation_1 = "problem Formulation_1: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R4;"
formulation_2 = "problem Formulation_2: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R34A, R34B, R34C, R4;"
formulation_3 = "problem Formulation_3: " + general_components + ", " + "natCountAll, R35, R36, R37, R38, R39, R33A, R33B, R33C, R4;"
formulation_4 = "problem Formulation_4: " + general_components + ", " + "natCountAll, R35, R36, R37, R38, R39, R33A, R33B, R33C, R34A, R34B, R34C, R4;"
formulation_5 = "problem Formulation_5: " + general_components + ", " + "R31, R32, R33A, R33B, R33C, R4B;"


def get_solved_elased_time_str(results):
    times = [r["solve_elapsed_time"] for r in results]
    median = np.median(times)
    std = np.std(times)
    return f"{median:.3f} ± ({std:.3f}) seconds"

### Trying formulations from 1 to 5

In [2]:
results = {}; squads = {}

formulation_name = "Formulation_1"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)
    
formulation_name = "Formulation_2"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_3"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_4"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_4", formulation_4, print_results=False, print_summary=False, n_repeats=10)

formulation_name = "Formulation_5"
results[formulation_name], squads[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_5", formulation_5, print_results=False, print_summary=False, n_repeats=10)

In [3]:
def get_solved_elased_time_str(results):
    times = [r["solve_elapsed_time"] for r in results]
    median = np.median(times)
    std = np.std(times)
    return f"{median:.3f} ± ({std:.3f}) seconds"

new_times = {}
for key in results.keys():
    new_times[key] = get_solved_elased_time_str(results[key])


summary_results = pd.DataFrame([results[key][0] for key in results.keys()], index=results.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_formulaciones.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1,solved,8580,8401,0.0,15839,113,1.531 ± (0.150) seconds,86.909091
Formulation_2,solved,8580,13438,0.0,19410,175,2.391 ± (0.173) seconds,86.909091
Formulation_3,solved,10259,13438,0.0,80983,451,15.015 ± (0.420) seconds,86.909091
Formulation_4,solved,10259,18475,0.0,94185,1422,18.445 ± (0.305) seconds,86.909091
Formulation_5,solved,8580,8401,0.0,35043,954,3.047 ± (0.034) seconds,86.909091


### Example of solution

In [2]:
formulation_name = "Formulation_1"
_, squads= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=1)
squad = squads[0]
display(squad)
price = squad['price'].sum()
mean_score = squad['global_score'].mean()
chemistry_nationality = calculate_chemistry_nationality(squad)
print(f"Price: {price}€, Mean score: {mean_score:.4f}, Nationality chemistry:{chemistry_nationality}")

,name,player_type,generic_position,position,price,global_score,nation,is_hero,is_icon
0,Yann Sommer,Normal,GK,GK,8900,87,47,False,False
1,Wendie Renard,Normal,DEF,CB,13500,88,18,False,False
2,Joshua Kimmich,Normal,DEF,RB,5200,86,21,False,False
3,Marcos Aoás Corrêa,Normal,DEF,CB,9200,87,54,False,False
4,Gabriel dos S. Magalhães,Normal,DEF,CB,5000,86,54,False,False
5,Florian Wirtz,Normal,MID,CAM,13250,88,21,False,False
6,Enzo Francescoli,Base Hero,MID,CAM,15000,88,60,True,False
7,İlkay Gündoğan,Normal,MID,CM,8900,87,21,False,False
8,Alexandra Popp,Normal,FOR,ST,9000,87,21,False,False
9,Karim Benzema,Normal,FOR,ST,5400,86,18,False,False


Price: 98550€, Mean score: 86.9091, Nationality chemistry:17


### Try another value of M

In [4]:
results_M = {}; squads_M = {}

params["M"] = 11
formulation_name = "Formulation_1_M11"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

params["M"] = 1867
formulation_name = "Formulation_1_M1867"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

params["M"] = 10^6
formulation_name = "Formulation_1_M10^6"
results_M[formulation_name], squads_M[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10)

In [5]:
new_times = {}
for key in results_M.keys():
    new_times[key] = get_solved_elased_time_str(results_M[key])


summary_results = pd.DataFrame([results_M[key][0] for key in results_M.keys()], index=results_M.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_M.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1_M11,solved,8580,8401,0.0,15839,113,1.571 ± (0.037) seconds,86.909091
Formulation_1_M1867,solved,8580,8401,0.0,39106,1182,4.680 ± (0.080) seconds,86.909091
Formulation_1_M10^6,solved,8580,8401,0.0,16523,120,1.594 ± (0.049) seconds,86.909091


### AMPL Gurobi cuts

In [6]:
results_cuts = {}; squads_cuts = {}
params["M"] = 11

formulation_name = "Formulation_1_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_1_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_1_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

formulation_name = "Formulation_2_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_2_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_2_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

formulation_name = "Formulation_3_cuts1"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=1)
formulation_name = "Formulation_3_cuts2"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=2)
formulation_name = "Formulation_3_cuts3"
results_cuts[formulation_name], squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_3", formulation_3, print_results=False, print_summary=False, n_repeats=10, cuts_param=3)

new_times = {}
for key in results_cuts.keys():
    new_times[key] = get_solved_elased_time_str(results_cuts[key])


summary_results = pd.DataFrame([results_cuts[key][0] for key in results_cuts.keys()], index=results_cuts.keys()).drop("output", axis=1)
summary_results["solve_elapsed_time"] = summary_results.index.map(new_times)
display(summary_results)
summary_results.to_latex("table_cuts.tex", float_format="%.3f", escape=False)

,solve_result,num_vars,num_constraints,gap,simplex_iterations,branching_nodes,solve_elapsed_time,objective
Formulation_1_cuts1,solved,8580,8401,0.0,15958,1,2.336 ± (0.051) seconds,86.909091
Formulation_1_cuts2,solved,8580,8401,0.0,14945,1,2.125 ± (0.093) seconds,86.909091
Formulation_1_cuts3,solved,8580,8401,0.0,15460,1,2.899 ± (0.144) seconds,86.909091
Formulation_2_cuts1,solved,8580,13438,0.0,18087,1,3.242 ± (0.049) seconds,86.909091
Formulation_2_cuts2,solved,8580,13438,0.0,16290,1,3.321 ± (0.120) seconds,86.909091
Formulation_2_cuts3,solved,8580,13438,0.0,16539,1,3.665 ± (0.093) seconds,86.909091
Formulation_3_cuts1,solved,10259,13438,0.0,19877,1,7.531 ± (0.141) seconds,86.909091
Formulation_3_cuts2,solved,10259,13438,0.0,19826,1,7.851 ± (0.058) seconds,86.909091
Formulation_3_cuts3,solved,10259,13438,0.0,28685,1,17.922 ± (0.249) seconds,86.909091


#### Let's see the specific cuts generated by Gurobi

In [14]:
_, _ = repeat_run_experiment(None, "players_1867.csv", params, "Formulation_1", formulation_1, print_results=True, print_summary=False, n_repeats=1, cuts_param=3)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
8401 constraints, all linear; 177799 nonzeros
	4 equality constraints
	8397 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 3
  cut:cuts = 3
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  3
InfUnbdInfo  1
Presolve  0

Optimize a model with 8401 rows, 8580 columns and 177799 nonzeros
Model fingerprint: 0x7915281b
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range    

In [15]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=1)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 1
  cut:cuts = 1
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  1
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 

In [16]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=2)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 2
  cut:cuts = 2
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  2
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 

In [17]:
_, _ = squads_cuts[formulation_name]= repeat_run_experiment(None, "players_1867.csv", params, "Formulation_2", formulation_2, print_results=True, print_summary=False, n_repeats=2, cuts_param=3)

Presolve eliminates 0 constraints and 740 variables.
Adjusted problem:
8580 variables:
	6901 binary variables
	1679 integer variables
13438 constraints, all linear; 187873 nonzeros
	4 equality constraints
	13434 inequality constraints
1 linear objective; 1864 nonzeros.

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter Presolve to value 0
  pre:solve = 0
Set parameter Cuts to value 3
  cut:cuts = 3
  mip:bestbound = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Cuts  3
InfUnbdInfo  1
Presolve  0

Optimize a model with 13438 rows, 8580 columns and 187873 nonzeros
Model fingerprint: 0x78bec64e
Variable types: 0 continuous, 8580 integer (0 binary)
Coefficient statistics:
  Matrix range 